## 2RPFS Problem (Cmax objective) - Tables and Graphs

Before running this, notebook, please run notebook 0.1.

In [1]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import seaborn as sns
import gzip
import matplotlib.style as style
from matplotlib.path import Path
from matplotlib.patches import BoxStyle

%matplotlib inline

In [2]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

In [3]:
linestyle_tuple = [
     ('dotted',                (0, (1, 1))),
     ('dashed',                (0, (5, 5))),
     ('densely dashed',        (0, (5, 1))),
     ('dashdotdotted',         (0, (3, 5, 1, 5, 1, 5))),
     ('densely dashdotdotted', (0, (3, 1, 1, 1, 1, 1))),

     ('dashdotted',            (0, (3, 5, 1, 5))),
     ('densely dashdotted',    (0, (3, 1, 1, 1))),
     
     ('loosely dashed',        (0, (5, 10))),
     ('loosely dashdotted',    (0, (3, 10, 1, 10))),
     

     ('loosely dashdotdotted', (0, (3, 10, 1, 10, 1, 10))),
     ('densely dotted',        (0, (1, 1))),
     ('loosely dotted',        (0, (1, 10)))]

In [4]:
# https://stackoverflow.com/questions/51483901/is-there-a-way-to-extend-the-solid-color-background-to-the-full-width-of-the-pag
class ExtendedTextBox(BoxStyle._Base):
    """
    An Extended Text Box that expands to the axes limits 
                        if set in the middle of the axes
    """

    def __init__(self, pad=0.3, width=500.):
        """
        width: 
            width of the textbox. 
            Use `ax.get_window_extent().width` 
                   to get the width of the axes.
        pad: 
            amount of padding (in vertical direction only)
        """
        self.width=width
        self.pad = pad
        super(ExtendedTextBox, self).__init__()

    def transmute(self, x0, y0, width, height, mutation_size):
        """
        x0 and y0 are the lower left corner of original text box
        They are set automatically by matplotlib
        """
        # padding
        pad = mutation_size * self.pad

        # we add the padding only to the box height
        height = height + 2.*pad
        # boundary of the padded box
        y0 = y0 - pad
        y1 = y0 + height
        _x0 = x0
        x0 = _x0 +width /2. - self.width/2.
        x1 = _x0 +width /2. + self.width/2.

        cp = [(x0, y0),
              (x1, y0), (x1, y1), (x0, y1),
              (x0, y0)]

        com = [Path.MOVETO,
               Path.LINETO, Path.LINETO, Path.LINETO,
               Path.CLOSEPOLY]

        path = Path(cp, com)

        return path

### List files in the result folder 

In [5]:
resultfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
rpfs_file = os.path.join(resultfolder, 'RPFS_TWCT_all_results.pkl.gz')

### Create the output folder 

In [6]:
outputfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
outputfolder_graph = os.path.join(os.getcwd(), 'results', 'consolidated', 'graphs')
outputfolder_table = os.path.join(os.getcwd(), 'results', 'consolidated', 'tables')
if not os.path.exists(outputfolder_graph):
    os.makedirs(outputfolder_graph)
if not os.path.exists(outputfolder_table):
    os.makedirs(outputfolder_table)
#print('Saving files on folder: ' + outputfolder)

### Process consolidated CSV result files

In [7]:
df_rpfs = pd.read_pickle(rpfs_file)  # Robust PFSP Budget solutions only
df_rpfs.drop(columns=['executionId'], inplace=True)
df_rpfs = df_rpfs.reset_index()

**Robust dataframe: calculating new fields.**

In [8]:
df_rpfs['optimal'] = df_rpfs['is_optimal'] & df_rpfs['validated'] & (df_rpfs['gap'] <= 1e-8)
df_rpfs['time_limit'] = 7200.0
df_rpfs['time_limit_2'] = 7200.0 * 2
df_rpfs['mp_total_time'] = (df_rpfs['n'] < 15).astype(int) * np.minimum(df_rpfs['mp_total_time'], df_rpfs['time_limit']) + (df_rpfs['n'] >= 15).astype(int) * np.minimum(df_rpfs['mp_total_time'], df_rpfs['time_limit_2'])
df_rpfs['time'] = df_rpfs['mp_total_time'] + df_rpfs['sp_total_time']
df_rpfs['gap'] = df_rpfs['gap'] * 100.0
df_rpfs['RobCost_worstcase'] = df_rpfs['wct_validation']
df_rpfs = df_rpfs.rename(columns={"budget_Gamma": "RobCost_Gamma"})

In [9]:
df_rpfs.tail(4)

,model,n,m,alpha,seq,RobCost_Gamma,instance_type,ub_name,instance_name,wct,...,gap,lb,cost,wct_validation,Gamma%,optimal,time_limit,time_limit_2,time,RobCost_worstcase
13830,wilson,10,5,50,10,35.0,ying,mip_separation,RB0105010_10_5_50_wct_inputs.txt,153276.0,...,1.664494e+00,150766.5,153276.0,0.0,70.0,False,7200.0,14400.0,7352.10,0.0
13831,wilson,10,5,50,10,40.0,ying,mip_separation,RB0105010_10_5_50_wct_inputs.txt,153276.0,...,-1.898786e-14,153276.0,153276.0,153276.0,80.0,True,7200.0,14400.0,6724.10,153276.0
13832,wilson,10,5,50,10,45.0,ying,mip_separation,RB0105010_10_5_50_wct_inputs.txt,153276.0,...,0.000000e+00,153276.0,153276.0,153276.0,90.0,True,7200.0,14400.0,4128.44,153276.0
13833,wilson,10,5,50,10,50.0,ying,mip_separation,RB0105010_10_5_50_wct_inputs.txt,153276.0,...,1.898786e-14,153276.0,153276.0,153276.0,100.0,True,7200.0,14400.0,720.45,153276.0


In [10]:
df_rpfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13834 entries, 0 to 13833
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   model                  13834 non-null  object 
 1   n                      13834 non-null  int64  
 2   m                      13834 non-null  int64  
 3   alpha                  13834 non-null  int64  
 4   seq                    13834 non-null  object 
 5   RobCost_Gamma          13834 non-null  float64
 6   instance_type          13834 non-null  object 
 7   ub_name                13834 non-null  object 
 8   instance_name          13834 non-null  object 
 9   wct                    13831 non-null  float64
 10  permutation            13834 non-null  object 
 11  time_spent             13834 non-null  float64
 12  time_to_best_sol       13834 non-null  float64
 13  mp_total_time          13834 non-null  float64
 14  sp_total_time          13833 non-null  float64
 15  it

### Checking the Robust PFSP Budget solutions dataframe

In [11]:
df_rpfs.head(2)

,model,n,m,alpha,seq,RobCost_Gamma,instance_type,ub_name,instance_name,wct,...,gap,lb,cost,wct_validation,Gamma%,optimal,time_limit,time_limit_2,time,RobCost_worstcase
0,liao-you,10,2,10,01,2.0,ying,mip_separation,RB0101001_10_2_10_wct_inputs.txt,64014.2,...,3.409849e-14,64014.2,64014.2,64014.2,10.0,True,7200.0,14400.0,11.89,64014.2
1,liao-you,10,2,10,01,4.0,ying,mip_separation,RB0101001_10_2_10_wct_inputs.txt,65550.0,...,0.000000e+00,65550.0,65550.0,65550.0,20.0,True,7200.0,14400.0,11.74,65550.0


# Tables

Obtain list of C&CG models, instance types

In [12]:
model_list = df_rpfs['model'].unique().tolist()
instance_type_list = df_rpfs['instance_type'].unique().tolist()
print(model_list)
print(instance_type_list)

['liao-you', 'manne', 'tba', 'ts2', 'ts3', 'wagner-wst2', 'wilson']
['ying']


Add a new column containing the instance size as string

In [13]:
df_temp = df_rpfs
(df_temp['n'].astype(str) + 'x' + df_temp['m'].astype(str)).unique()

array(['10x2', '10x3', '10x4', '10x5', '15x5', '20x5'], dtype=object)

In [14]:
df_rpfs.columns

Index(['model', 'n', 'm', 'alpha', 'seq', 'RobCost_Gamma', 'instance_type',
       'ub_name', 'instance_name', 'wct', 'permutation', 'time_spent',
       'time_to_best_sol', 'mp_total_time', 'sp_total_time', 'iterations',
       'num_visited_solutions', 'num_improvements', 'is_optimal', 'validated',
       'gap', 'lb', 'cost', 'wct_validation', 'Gamma%', 'optimal',
       'time_limit', 'time_limit_2', 'time', 'RobCost_worstcase'],
      dtype='object')

In [15]:
df_temp = df_rpfs
df_temp['instance_size'] = df_temp['n'].astype(str) + 'x' + df_temp['m'].astype(str)
df_rpfs = df_temp.set_index(['model', 'n', 'm', 'alpha', 'seq', 'RobCost_Gamma', 'instance_type'])
df_rpfs

ub_name  \
model    n  m alpha seq RobCost_Gamma instance_type                   
liao-you 10 2 10    01  2.0           ying           mip_separation   
                        4.0           ying           mip_separation   
                        6.0           ying           mip_separation   
                        8.0           ying           mip_separation   
                        10.0          ying           mip_separation   
...                                                             ...   
wilson   10 5 50    10  30.0          ying           mip_separation   
                        35.0          ying           mip_separation   
                        40.0          ying           mip_separation   
                        45.0          ying           mip_separation   
                        50.0          ying           mip_separation   

                                                                        instance_name  \
model    n  m alpha seq RobCost_Gamma instance_type                                     
liao-you 10 2 10    01  2.0           ying           RB0101001_10_2_10_wct_inputs.txt   
                        4.0           ying           RB0101001_10_2_10_wct_inputs.txt   
                        6.0           ying           RB0101001_10_2_10_wct_inputs.txt   
                        8.0           ying           RB0101001_10_2_10_wct_inputs.txt   
                        10.0          ying           RB0101001_10_2_10_wct_inputs.txt   
...                                                                               ...   
wilson   10 5 50    10  30.0          ying           RB0105010_10_5_50_wct_inputs.txt   
                        35.0          ying           RB0105010_10_5_50_wct_inputs.txt   
                        40.0          ying           RB0105010_10_5_50_wct_inputs.txt   
                        45.0          ying           RB0105010_10_5_50_wct_inputs.txt   
                        50.0          ying           RB0105010_10_5_50_wct_inputs.txt   

                                                          wct  \
model    n  m alpha seq RobCost_Gamma instance_type             
liao-you 10 2 10    01  2.0           ying            64014.2   
                        4.0           ying            65550.0   
                        6.0           ying            66175.1   
                        8.0           ying            66617.5   
                        10.0          ying            67028.2   
...                                                       ...   
wilson   10 5 50    10  30.0          ying           153276.0   
                        35.0          ying           153276.0   
                        40.0          ying           153276.0   
                        45.0          ying           153276.0   
                        50.0          ying           153276.0   

                                                              permutation  \
model    n  m alpha seq RobCost_Gamma instance_type                         
liao-you 10 2 10    01  2.0           ying           7 9 1 6 10 4 3 8 5 2   
                        4.0           ying           7 9 1 6 10 4 3 8 5 2   
                        6.0           ying           7 9 1 6 10 4 3 8 5 2   
                        8.0           ying           7 9 1 6 10 4 3 8 5 2   
                        10.0          ying           7 9 1 6 10 4 3 8 5 2   
...                                                                   ...   
wilson   10 5 50    10  30.0          ying           10 9 7 6 1 2 5 8 3 4   
                        35.0          ying           10 9 7 6 1 2 5 8 3 4   
                        40.0          ying           10 9 7 6 1 2 5 8 3 4   
                        45.0          ying           10 9 7 6 1 2 5 8 3 4   
                        50.0          ying           10 9 7 6 1 2 5 8 3 4   

                                                     time_spent  \
model    n  m alpha seq RobCost_Gamma instance_type            

Treating errors in the `gap` column

In [16]:
df_rpfs['gap'].describe()

count    1.373500e+04
mean     1.952817e-01
std      7.759377e-01
min     -3.361379e-01
25%      0.000000e+00
50%      1.574525e-14
75%      6.248000e-14
max      1.765520e+01
Name: gap, dtype: float64

In [17]:
df_rpfs['gap'] = df_rpfs['gap'].apply(lambda x: np.maximum(x, 0.0))

In [18]:
df_rpfs['gap'].describe()

count    1.373500e+04
mean     1.954066e-01
std      7.758810e-01
min      0.000000e+00
25%      0.000000e+00
50%      1.574525e-14
75%      6.248000e-14
max      1.765520e+01
Name: gap, dtype: float64

## Table 2. Performance given all instances 

Model-wise Robust PFSP C&CG performance comparison, given all instances.

* % Best Performance is the percentage of instances solved to optimality where the model achieved shorter execution time, when compared to the other models; 

* % Solved contains the percentage of instances solved within the time limit; 

* % Solved < n x m > represents the percentage of solved instances of size n x m; 

* Avg. % Gap is the average percentage gap of solutions from instances not solved to optimality; 

* Median time is the median execution time, in seconds; 

* Median iterations is the median of the number of iterations performed.

In [19]:
def calculate_perc_best_performance(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    df_model = df_model[df_model['optimal'] == True]    
    df_model = df_model.set_index(['n', 'm', 'alpha', 'seq', 'RobCost_Gamma', 'instance_type'])
    if len(df_model.index) == 0:
        return np.nan
    
    df_others = df.reset_index()
    df_others = df_others[df_others['model'] != model]
    df_others = df_others[df_others['optimal'] == True]   
    df_best_performance = df_others.groupby(by=['n', 'm', 'alpha', 'seq', 'RobCost_Gamma', 'instance_type']).min()['time']
    df_best_performance = df_best_performance.to_frame()
    if len(df_best_performance.index) == 0:
        return np.nan
    
    df_compare = df_best_performance.join(df_model, how='inner', 
                                                     on=['n', 'm', 'alpha', 'seq', 'RobCost_Gamma', 'instance_type'],
                                                     lsuffix='_best')
    df_compare['time_wins'] = (df_compare['time'] < df_compare['time_best']).astype(int)
    return np.round(100.0 * df_compare['time_wins'].sum() / len(df_compare.index), 2)

In [20]:
def calculate_perc_solved(df, model, instance_type = None, instance_size = None):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    df_ = df_model
    if instance_type is not None:
        df_ = df_[df_['instance_type'] == instance_type]
    if instance_size is not None:
        df_ = df_[df_['instance_size'] == instance_size]
    if len(df_.index) > 0:
        return np.round(100.0 * len(df_[(df_['optimal'] == True)].index) / len(df_.index), 2)
    else:
        return np.nan

In [21]:
# Avg. % Gap is the average percentage gap of solutions from instances not solved to optimality
def calculate_avg_perc_gap(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    df_model = df_model[df_model['optimal'] == False]
    return np.round(df_model['gap'].mean(), 2)

In [22]:
def calculate_median_time(df, model, time_col_name):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model[time_col_name].median(), 2)

In [23]:
def calculate_avg_time(df, model, time_col_name):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model[time_col_name].mean(), 2)

In [24]:
def calculate_std_time(df, model, time_col_name):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model[time_col_name].std(), 2)

In [25]:
def calculate_median_iterations(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model['iterations'].median(), 2)

In [26]:
def calculate_avg_iterations(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model['iterations'].mean(), 2)

In [27]:
def calculate_std_iterations(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model['iterations'].std(), 2)

In [28]:
model_stats = dict()
for model in model_list:
    model_stats[model] = dict()
    model_stats[model]['% Best Performance'] = calculate_perc_best_performance(df_rpfs, model)
    model_stats[model]['% Solved'] = calculate_perc_solved(df_rpfs, model)  # given all instances
    for instance_type in instance_type_list:  # group by instance type and size
        df_itype = df_rpfs.reset_index()
        df_itype = df_itype[(df_itype['instance_type'] == instance_type)]
        instance_size_list = df_itype['instance_size'].unique().tolist()
        for instance_size in instance_size_list:
            model_stats[model]['% Solved '+instance_type+' '+ instance_size] = calculate_perc_solved(df_rpfs, model, instance_type, instance_size)
    model_stats[model]['Avg. % gap'] = calculate_avg_perc_gap(df_rpfs, model)
    model_stats[model]['Median time'] = calculate_median_time(df_rpfs, model, 'time')
    model_stats[model]['Median time MP'] = calculate_median_time(df_rpfs, model, 'mp_total_time')
    model_stats[model]['Median time SP'] = calculate_median_time(df_rpfs, model, 'sp_total_time')
    model_stats[model]['Median iterations'] = calculate_median_iterations(df_rpfs, model)

In [29]:
model_stats

{'liao-you': {'% Best Performance': 4.2,
  '% Solved': 91.16,
  '% Solved ying 10x2': 100.0,
  '% Solved ying 10x3': 96.8,
  '% Solved ying 10x4': 84.0,
  '% Solved ying 10x5': 86.0,
  '% Solved ying 15x5': 7.69,
  '% Solved ying 20x5': nan,
  'Avg. % gap': 0.86,
  'Median time': 262.26,
  'Median time MP': 161.09,
  'Median time SP': 12.38,
  'Median iterations': 5.0},
 'manne': {'% Best Performance': 90.31,
  '% Solved': 93.38,
  '% Solved ying 10x2': 100.0,
  '% Solved ying 10x3': 100.0,
  '% Solved ying 10x4': 97.05,
  '% Solved ying 10x5': 98.4,
  '% Solved ying 15x5': 12.0,
  '% Solved ying 20x5': 0.0,
  'Avg. % gap': 1.49,
  'Median time': 87.3,
  'Median time MP': 39.95,
  'Median time SP': 7.83,
  'Median iterations': 4.0},
 'tba': {'% Best Performance': 0.07,
  '% Solved': 69.08,
  '% Solved ying 10x2': 91.2,
  '% Solved ying 10x3': 73.65,
  '% Solved ying 10x4': 59.6,
  '% Solved ying 10x5': 52.26,
  '% Solved ying 15x5': nan,
  '% Solved ying 20x5': nan,
  'Avg. % gap': 1.0

In [30]:
model_stats_df = pd.DataFrame.from_dict(model_stats)
model_stats_df

,liao-you,manne,tba,ts2,ts3,wagner-wst2,wilson
% Best Performance,4.20,90.31,0.07,0.00,2.33,0.00,4.35
% Solved,91.16,93.38,69.08,79.23,76.80,81.19,69.95
% Solved ying 10x2,100.00,100.00,91.20,99.20,90.80,99.20,95.40
% Solved ying 10x3,96.80,100.00,73.65,77.34,76.34,82.40,74.65
% Solved ying 10x4,84.00,97.05,59.60,63.87,68.60,65.60,51.68
% Solved ying 10x5,86.00,98.40,52.26,74.84,71.83,74.83,60.56
% Solved ying 15x5,7.69,12.00,NaN,NaN,0.00,NaN,NaN
% Solved ying 20x5,NaN,0.00,NaN,NaN,NaN,NaN,NaN
Avg. % gap,0.86,1.49,1.03,1.16,0.69,1.16,1.09
Median time,262.26,87.30,1343.56,965.36,367.70,740.52,784.41


In [31]:
# Save output table as HTML
pd.set_option('colheader_justify', 'center')   # FOR TABLE <th>

html_string = '''
<html>
  <head><title>HTML Pandas Dataframe with CSS</title></head>
  <link rel="stylesheet" type="text/css" href="df_style.css"/>
  <body>
    {table}
  </body>
</html>.
'''

# OUTPUT AN HTML FILE
with open(os.path.join(outputfolder_table, 'model_stats.html'), 'w') as f:
    f.write(html_string.format(table=model_stats_df.to_html(classes='mystyle')))

## Table 3. Performance per instance group and model

Model-wise Robust PFSP C&CG performance comparison, per instance group.

* % Best Performance is the percentage of instances solved to optimality where the model achieved shorter execution time, when compared to the other models; 

* % Solved contains the percentage of instances solved within the time limit; 

* Avg. % Gap is the average percentage gap of solutions from instances not solved to optimality; 

* Avg. time and Std. dev. of time are the mean and standard deviation in solution time (s), respectively;

* Avg. iterations and Std. dev. of iterations are the mean and standard deviation of the number of iterations performed.

In [32]:
per_instance_stats = dict()
for instance_type in instance_type_list:  # group by instance type and size
    df_itype = df_rpfs.reset_index()
    df_itype = df_itype[(df_itype['instance_type'] == instance_type)]
    instance_size_list = df_itype['instance_size'].unique().tolist()
    for instance_size in instance_size_list:
        df_instance = df_itype[df_itype['instance_size'] == instance_size]
        for model in model_list:
            per_instance_stats[(instance_type,instance_size,model)] = dict()
            per_instance_stats[(instance_type,instance_size,model)]['% Best Performance'] = calculate_perc_best_performance(df_instance, model)
            per_instance_stats[(instance_type,instance_size,model)]['% Solved'] = calculate_perc_solved(df_rpfs, model, instance_type, instance_size)
            per_instance_stats[(instance_type,instance_size,model)]['Avg. % gap'] = calculate_avg_perc_gap(df_instance, model)
            per_instance_stats[(instance_type,instance_size,model)]['Avg. time'] = calculate_avg_time(df_instance, model, 'time')
            per_instance_stats[(instance_type,instance_size,model)]['Std. dev. of time'] = calculate_std_time(df_instance, model, 'time')
            per_instance_stats[(instance_type,instance_size,model)]['Avg. MP time'] = calculate_avg_time(df_instance, model, 'mp_total_time')
            per_instance_stats[(instance_type,instance_size,model)]['Avg. SP time'] = calculate_avg_time(df_instance, model, 'sp_total_time')
            per_instance_stats[(instance_type,instance_size,model)]['Avg. iterations'] = calculate_avg_iterations(df_instance, model)
            per_instance_stats[(instance_type,instance_size,model)]['Std. dev. of iterations'] = calculate_std_iterations(df_instance, model)

In [33]:
per_instance_stats

{('ying', '10x2', 'liao-you'): {'% Best Performance': 6.0,
  '% Solved': 100.0,
  'Avg. % gap': nan,
  'Avg. time': 158.35,
  'Std. dev. of time': 422.59,
  'Avg. MP time': 157.81,
  'Avg. SP time': 0.54,
  'Avg. iterations': 3.76,
  'Std. dev. of iterations': 2.2},
 ('ying', '10x2', 'manne'): {'% Best Performance': 81.4,
  '% Solved': 100.0,
  'Avg. % gap': nan,
  'Avg. time': 35.34,
  'Std. dev. of time': 62.46,
  'Avg. MP time': 33.37,
  'Avg. SP time': 1.97,
  'Avg. iterations': 3.25,
  'Std. dev. of iterations': 1.62},
 ('ying', '10x2', 'tba'): {'% Best Performance': 0.0,
  '% Solved': 91.2,
  'Avg. % gap': 0.12,
  'Avg. time': 792.93,
  'Std. dev. of time': 2018.5,
  'Avg. MP time': 791.36,
  'Avg. SP time': 1.57,
  'Avg. iterations': 14.79,
  'Std. dev. of iterations': 41.37},
 ('ying', '10x2', 'ts2'): {'% Best Performance': 0.0,
  '% Solved': 99.2,
  'Avg. % gap': 0.51,
  'Avg. time': 529.34,
  'Std. dev. of time': 1145.15,
  'Avg. MP time': 529.05,
  'Avg. SP time': 0.29,
  'A

In [34]:
# https://stackoverflow.com/questions/57606801/pandas-style-options-to-latex

In [35]:
pd.set_option('display.max_columns', None)
allowed_keys = [(x, y, z) for (x, y, z) in per_instance_stats.keys() if (x == 'ying' and y in ['10x2', '10x3', '10x4'])]
per_instance_stats1 = { your_key: per_instance_stats[your_key] for your_key in allowed_keys }
df_table3a = pd.DataFrame.from_dict(per_instance_stats1)
df_table3a

ying                                       \
                          10x2                                        
                        liao-you  manne    tba      ts2      ts3      
% Best Performance         6.00    81.40     0.00     0.00     0.00   
% Solved                 100.00   100.00    91.20    99.20    90.80   
Avg. % gap                  NaN      NaN     0.12     0.51     0.12   
Avg. time                158.35    35.34   792.93   529.34   800.66   
Std. dev. of time        422.59    62.46  2018.50  1145.15  2058.42   
Avg. MP time             157.81    33.37   791.36   529.05   798.22   
Avg. SP time               0.54     1.97     1.57     0.29     2.44   
Avg. iterations            3.76     3.25    14.79     3.24    12.97   
Std. dev. of iterations    2.20     1.62    41.37     1.63    32.73   

                                                                        \
                                               10x3                      
                        wagner-wst2  wilson  liao-you  manne    tba      
% Best Performance            0.00     13.21     2.89   93.44     0.00   
% Solved                     99.20     95.40    96.80  100.00    73.65   
Avg. % gap                    0.20      0.29     0.30     NaN     0.51   
Avg. time                   469.31    435.42   837.40  183.66  2916.55   
Std. dev. of time          1000.78   1498.13  1625.65  347.83  2945.74   
Avg. MP time                469.03    433.84   833.61  179.92  2909.15   
Avg. SP time                  0.27      1.58     3.79    3.74     7.40   
Avg. iterations               3.75     13.32     5.90    5.43     7.57   
Std. dev. of iterations       2.16     55.50     4.17    3.60     9.42   

                                                                         \
                                                                 10x4     
                          ts2      ts3    wagner-wst2  wilson  liao-you   
% Best Performance          0.00     4.69       0.00      0.00     5.24   
% Solved                   77.34    76.34      82.40     74.65    84.00   
Avg. % gap                  0.88     0.46       0.87      0.48     0.86   
Avg. time                3059.47  2081.42    2504.75   2565.03  2128.38   
Std. dev. of time        3011.68  2999.21    2862.08   3041.40  2710.47   
Avg. MP time             3048.99  2072.43    2502.45   2554.22  2021.14   
Avg. SP time               10.48     8.99       2.30     10.81   107.24   
Avg. iterations             4.62    21.85       5.06      6.43     7.90   
Std. dev. of iterations     2.12    43.72       2.63      4.61     5.65   

                                                                         \
                                                                          
                         manne     tba      ts2      ts3    wagner-wst2   
% Best Performance         92.81     0.00     0.00     1.17       0.00    
% Solved                   97.05    59.60    63.87    68.60      65.60    
Avg. % gap                  0.68     1.29     1.62     0.99       1.57    
Avg. time                 781.21  4170.72  3978.71  3108.75    3634.75    
Std. dev. of time        1556.30  3062.56  3059.35  3163.55    3070.96    
Avg. MP time              667.59  4086.18  3889.88  2943.27    3588.78    
Avg. SP time              113.62    84.54    88.83   165.49      45.97    
Avg. iterations             7.69     7.21     5.71    10.92       5.76    
Std. dev. of iterations     5.77     6.20     2.84    13.83       2.93    

                                  
                                  
                         wilson   
% Best Performance          0.00  
% Solved                   51.68  
Avg. % gap                  1.40  
Avg. time                4403.42  
Std. dev. of time        3223.47  
Avg. MP time             4283.44  
Avg. SP time              124.94  
Avg. iterations             6.81  
Std. dev. of iterations     3.75

In [36]:
pd.set_option('display.max_columns', None)
allowed_keys = [(x, y, z) for (x, y, z) in per_instance_stats.keys() if (x == 'ying' and y in ['10x5', '15x5'])]
per_instance_stats2 = { your_key: per_instance_stats[your_key] for your_key in allowed_keys }
df_table3b = pd.DataFrame.from_dict(per_instance_stats2)
df_table3b

ying                                        \
                          10x5                                         
                        liao-you  manne     tba      ts2      ts3      
% Best Performance          2.56    94.27     0.38     0.00     3.87   
% Solved                   86.00    98.40    52.26    74.84    71.83   
Avg. % gap                  0.40     0.60     1.25     0.55     0.70   
Avg. time                2116.04   713.29  4625.81  3192.31  2990.09   
Std. dev. of time        2633.53  1221.27  3263.92  3109.28  3224.48   
Avg. MP time             1543.03   338.12  4090.66  2664.21  2359.08   
Avg. SP time              573.01   375.17   535.14   528.10   631.01   
Avg. iterations             8.04     6.73     5.31     5.23     9.47   
Std. dev. of iterations     8.07     5.31     3.02     3.54    14.55   

                                                                          \
                                                15x5                       
                        wagner-wst2  wilson   liao-you   manne   tba ts2   
% Best Performance            0.00      0.00      0.00    100.00 NaN NaN   
% Solved                     74.83     60.56      7.69     12.00 NaN NaN   
Avg. % gap                    0.58      1.15      4.29      2.46 NaN NaN   
Avg. time                  3610.84   3991.79  13574.11  12861.37 NaN NaN   
Std. dev. of time          3126.35   3327.95   3267.65   4471.30 NaN NaN   
Avg. MP time               3025.24   3520.47  13497.57  12793.52 NaN NaN   
Avg. SP time                585.60    471.32     76.54     67.86 NaN NaN   
Avg. iterations               4.85      5.92      3.38      3.04 NaN NaN   
Std. dev. of iterations       3.00      3.53      0.96      1.46 NaN NaN   

                                                      
                                                      
                           ts3    wagner-wst2 wilson  
% Best Performance            NaN     NaN      NaN    
% Solved                     0.00     NaN      NaN    
Avg. % gap                   5.20     NaN      NaN    
Avg. time                24526.66     NaN      NaN    
Std. dev. of time        17662.88     NaN      NaN    
Avg. MP time             12101.20     NaN      NaN    
Avg. SP time             12425.46     NaN      NaN    
Avg. iterations              1.00     NaN      NaN    
Std. dev. of iterations      0.00     NaN      NaN

In [38]:
# Save output table as HTML
pd.set_option('colheader_justify', 'center')   # FOR TABLE <th>

html_string = '''
<html>
  <head><title>HTML Pandas Dataframe with CSS</title></head>
  <link rel="stylesheet" type="text/css" href="df_style.css"/>
  <body>
    {table}
  </body>
</html>.
'''

# OUTPUT AN HTML FILE
with open(os.path.join(outputfolder_table, 'instance_stats_1.html'), 'w') as f:
    f.write(html_string.format(table=df_table3a.to_html(classes='mystyle')))
with open(os.path.join(outputfolder_table, 'instance_stats_2.html'), 'w') as f:
    f.write(html_string.format(table=df_table3b.to_html(classes='mystyle')))